In [1]:
import pandas as pd
import numpy as np
import re
import itertools
import matplotlib.pyplot as plt

In [2]:
#filename = open(r"/Users/pavanpandya/Desktop/HomeWork/Untitled Folder 1/steam.csv", "r")
filename = open(r"/Users/pavanpandya/Desktop/IDS_CIS_563/Project/clean_dataset.csv", "r")

df = pd.read_csv(filename,  error_bad_lines=False, encoding='utf-8')
#These are just quick checks to make sure the dataset looks correct
print(df.shape)
df.head()

(27075, 18)


/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/1784312750.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(filename,  error_bad_lines=False, encoding='utf-8')


,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [3]:
#Which columns have null values?print(df.columns[df.isna().any()].tolist())
#How many null values per column? - Count the missing values in each column
df.isnull().sum()

appid               0
name                0
release_date        0
english             0
developer           0
publisher           0
platforms           0
required_age        0
categories          0
genres              0
steamspy_tags       0
achievements        0
positive_ratings    0
negative_ratings    0
average_playtime    0
median_playtime     0
owners              0
price               0
dtype: int64

In [4]:
def extract_year(date):
    year = date[:4]
    if year.isnumeric():
        return int(year)
    else:
        return np.nan
df['year'] = df['release_date'].apply(extract_year)

In [5]:
#Create score column
def create_score(row):
    pos_count = row['positive_ratings']
    neg_count = row['negative_ratings']
    total_count = pos_count + neg_count
    average = pos_count / total_count
    return round(average, 2)
def total_ratings(row):
    pos_count = row['positive_ratings']
    neg_count = row['negative_ratings']
    total_count = pos_count + neg_count
    return total_count
df['total_ratings'] = df.apply(total_ratings, axis=1)
df['score'] = df.apply(create_score, axis=1)

In [6]:
# Calculate mean of vote average column
C = df['score'].mean()
# Calculate the minimum number of votes required to be in the chart
m = df['total_ratings'].quantile(0.90)
print(m)

908.6000000000022


In [7]:
# calculate the weighted rating for each qualified game
# Function that computes the weighted rating of each game
def weighted_rating(x, m=m, C=C):
    v = x['total_ratings']
    R = x['score']
    # Calculation based on the IMDB formula
    return round((v/(v+m) * R) + (m/(m+v) * C), 2)
# Define a new feature 'score' and calculate its value with `weighted_rating()`
df['weighted_score'] = df.apply(weighted_rating, axis=1)
#Print the top 15 games
df[['name', 'total_ratings', 'score', 'weighted_score']].head(15)

,name,total_ratings,score,weighted_score
0,Counter-Strike,127873,0.97,0.97
1,Team Fortress Classic,3951,0.84,0.82
2,Day of Defeat,3814,0.90,0.86
3,Deathmatch Classic,1540,0.83,0.79
4,Half-Life: Opposing Force,5538,0.95,0.92
5,Ricochet,3442,0.80,0.78
6,Half-Life,28855,0.96,0.95
7,Counter-Strike: Condition Zero,13559,0.89,0.88
8,Half-Life: Blue Shift,4242,0.90,0.87
9,Half-Life 2,70321,0.97,0.97


In [8]:
#We're adding this is for tags with multiple words, we need to connect by '-' before we split them by ' '
df['steamspy_tags'] = df['steamspy_tags'].str.replace(' ','-')
#TF-IDF Vectorizer further down will identify the words by the spaces between the words
df['genres'] = df['steamspy_tags'].str.replace(';',' ')
# count the number of occurences for each genre in the data set
counts = dict()
for i in df.index:
#for each element in list (each row, split by ' ', in genres column)
#-- we're splitting by space so tfidf can interpret the rows
    for j in df.loc[i,'genres'].split(' '):
    #if element is not in counts(dictionary of genres)
        if j not in counts:
            #give genre dictonary entry the value of 1
            counts[j] = 1
        else:
            #increase genre dictionary entry by 1
            counts[j] = counts[j] + 1
#Test Genre Counts
counts.keys()
print(counts['Action'])

10322


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
# create an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# apply the object to the genres column
# convert the list of documents (rows of genre tags) into a matrix
tfidf_matrix = tfidf_vector.fit_transform(df['genres'])

In [11]:
tfidf_matrix.shape

(27075, 370)

In [12]:
print(list(enumerate(tfidf_vector.get_feature_names())))

[(0, '1980s'), (1, '1990'), (2, '2d'), (3, '360'), (4, '3d'), (5, '40k'), (6, '4x'), (7, '5d'), (8, '6dof'), (9, 'abstract'), (10, 'access'), (11, 'action'), (12, 'adventure'), (13, 'agriculture'), (14, 'aliens'), (15, 'alternate'), (16, 'america'), (17, 'animation'), (18, 'anime'), (19, 'apocalyptic'), (20, 'arcade'), (21, 'arena'), (22, 'arts'), (23, 'assassin'), (24, 'atmospheric'), (25, 'attack'), (26, 'audio'), (27, 'awkward'), (28, 'base'), (29, 'baseball'), (30, 'based'), (31, 'basketball'), (32, 'batman'), (33, 'battle'), (34, 'beat'), (35, 'beautiful'), (36, 'benchmark'), (37, 'bikes'), (38, 'blood'), (39, 'bmx'), (40, 'board'), (41, 'book'), (42, 'bowling'), (43, 'builder'), (44, 'building'), (45, 'bullet'), (46, 'capitalism'), (47, 'card'), (48, 'cartoon'), (49, 'cartoony'), (50, 'casual'), (51, 'cats'), (52, 'character'), (53, 'chess'), (54, 'choices'), (55, 'choose'), (56, 'cinematic'), (57, 'city'), (58, 'class'), (59, 'classic'), (60, 'click'), (61, 'clicker'), (62, 'col

/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# create the cosine similarity matrix
sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)
print(sim_matrix)

[[1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [0.53785954 0.53785954 1.         ... 0.         0.         0.        ]
 ...
 [0.16328518 0.16328518 0.         ... 1.         0.61573349 0.61573349]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]]


In [14]:
def matching_score(a,b):
   return fuzz.ratio(a,b)
   # exactly the same, the score becomes 100

In [15]:
def get_title_year_from_index(index):
   return df[df.index == index]['year'].values[0]
def get_title_from_index(index):
   return df[df.index == index]['name'].values[0]
def get_index_from_title(title):
   return df[df.name == title].index.values[0]
def get_score_from_index(index):
   return df[df.index == index]['score'].values[0]
def get_weighted_score_from_index(index):
   return df[df.index == index]['weighted_score'].values[0]
def get_total_ratings_from_index(index):
   return df[df.index == index]['total_ratings'].values[0]
def get_platform_from_index(index):
   return df[df.index == index]['platforms'].values[0]

In [16]:
def find_closest_title(title):
    leven_scores = list(enumerate(df['name'].apply(matching_score, b=title))) #[(0, 30), (1,95), (2, 19)~~] A tuple of distances per index
    sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True) #Sorts list of tuples by distance [(1, 95), (3, 49), (0, 30)~~]
    closest_title = get_title_from_index(sorted_leven_scores[0][0])
    distance_score = sorted_leven_scores[0][1]
    return closest_title, distance_score
# Bejeweled Twist, 100

In [17]:
#find_closest_title returns only one title but I want a dropdown of the 10 closest game titles
def closest_names(title):
    leven_scores = list(enumerate(df['name'].apply(matching_score, b=title)))
    sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
    top_closest_names = [get_title_from_index(i[0]) for i in sorted_leven_scores[:10]]  #[['Team Fortress Classic', 'Deathmatch Classic', 'Counter-Strike',~~]
    return top_closest_names
closest_names('Call of Duty')

['Call of Duty®',
 'Call of Duty® 2',
 'Call of Duty®: WWII',
 'Call of Duty®: Ghosts',
 'Call of War',
 'Call of Juarez™',
 'Call of Duty®: Black Ops',
 'Call of Bitcoin',
 'Call of Cthulhu®',
 'Ball of Light']

In [18]:
# TA/Sir Please Install this library if it is not installed in your system , thanks
#!pip install gradio
import gradio as gr

In [19]:
def gradio_contents_based_recommender_v2(game, how_many, dropdown_option, sort_option, min_year, platform, min_score):
    #Return closest game title match
    closest_title, distance_score = find_closest_title(dropdown_option)
    #Create a Dataframe with these column headers
    recomm_df = pd.DataFrame(columns=['Game Title', 'Year', 'Score', 'Weighted Score', 'Total Ratings'])
    #Make the closest title whichever dropdown option the user has chosen
    closest_title = dropdown_option
    #find the corresponding index of the game title
    games_index = get_index_from_title(closest_title)
    #return a list of the most similar game indexes as a list
    games_list = list(enumerate(sim_matrix[int(games_index)]))
    #Sort list of similar games from top to bottom
    similar_games = list(filter(lambda x:x[0] != int(games_index), sorted(games_list,key=lambda x:x[1], reverse=True)))
    #Print the game title the similarity matrix is based on
    print('Here\'s the list of games similar to '+'\033[1m'+str(closest_title)+'\033[0m'+'.\n')
    #Only return the games that are on selected platform
    n_games = []
    for i,s in similar_games:
        if platform in get_platform_from_index(i):
            n_games.append((i,s))
    #Only return the games that are above the minimum score
    high_scores = []
    for i,s in n_games:
        if get_score_from_index(i) > min_score:
            high_scores.append((i,s))
    #Return the game tuple (game index, game distance score) and store in a dataframe
    for i,s in n_games[:how_many]:
        #Dataframe will contain attributes based on game index
        row = {'Game Title': get_title_from_index(i), 'Year': get_title_year_from_index(i), 'Score': get_score_from_index(i), 'Weighted Score': get_weighted_score_from_index(i), 'Total Ratings': get_total_ratings_from_index(i),}
        #Append each row to this dataframe
        recomm_df = recomm_df.append(row, ignore_index = True)
    #Sort dataframe by Sort_Option provided by userrecomm_df = recomm_df.sort_values(sort_option, ascending=False)
    #Only include games released same or after minimum year selectedrecomm_df = recomm_df[recomm_df['Year'] >= min_year]
    return recomm_df

In [ ]:
#Create list of unique calendar years based on main df column
years_sorted = sorted(list(df['year'].unique()))
#Ask user for input
print("What games do you want most similar to?:")
names = closest_names(input())
#Interface will include these buttons based on parameters in the function with a dataframe output
dropdown = gr.Interface(gradio_contents_based_recommender_v2, ["text", gr.inputs.Slider(1, 20, step=int(1)), gr.inputs.Dropdown(names), gr.inputs.Radio(['Year','Score','Weighted Score','Total Ratings']), gr.inputs.Slider(int(years_sorted[0]), int(years_sorted[-1]), step=int(1)), gr.inputs.Radio(['windows','xbox','playstation','linux','mac']), gr.inputs.Slider(0, 10, step=0.1)], "dataframe")
dropdown.launch(debug=True)

What games do you want most similar to?:
Counter-Strike: Global Offensive


/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/gradio/inputs.py:88: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/gradio/inputs.py:216: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/pavanpandya/opt/anaconda3/lib/python3.9/site-packages/gradio/inputs.py:182: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Here's the list of games similar to Counter-Strike: Global Offensive.



/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)
/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)
/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)


Here's the list of games similar to Counter-Strike: Source.



/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)
/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)
/var/folders/62/5fx30fbj1k38hrcxln9d4bwr0000gn/T/ipykernel_54930/2769069571.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recomm_df = recomm_df.append(row, ignore_index = True)
